In [3]:
# ========================
# 第一部分: 安装依赖
# ========================
!pip install -U langchain langchain-community faiss-cpu sentence-transformers
!pip install tqdm
print("\n所有必要的依赖库都已安装完毕。")



所有必要的依赖库都已安装完毕。


In [1]:
import os
import time
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from google.colab import drive
from tqdm import tqdm

def load_and_tag_documents(knowledge_base_path):
    print(f"\n 开始从您的Google Drive加载并分类知识水晶...")
    all_tagged_docs = []
    category_map = {
        "ships": "角色人设", "Character": "个人剧情", "Daily": "日常剧情",
        "EX": "活动剧情", "SP": "港区剧情"
    }
    for folder_name, category_name in category_map.items():
        directory = os.path.join(knowledge_base_path, folder_name)
        if not os.path.isdir(directory):
            print(f"  -> 警告：找不到文件夹 '{directory}'，已跳过。")
            continue
        loader = DirectoryLoader(
            directory, glob="**/*.md", loader_cls=TextLoader,
            loader_kwargs={'encoding': 'utf-8'}, show_progress=True
        )
        docs = loader.load()
        for doc in docs:
            doc.metadata['category'] = category_name
        all_tagged_docs.extend(docs)
        print(f"  -> 从 '{folder_name}' 加载了 {len(docs)} 份档案，并标记为【{category_name}】！")
    if not all_tagged_docs:
        print("\n 没有加载到任何文件")
    else:
        print(f"\n总共加载并标记了 {len(all_tagged_docs)} 份档案！加载完成！♪")
    return all_tagged_docs

def create_and_save_vector_store_with_progress(docs, save_path):
    """
    使用分批处理和tqdm进度条，来构建并保存向量数据库。
    """
    if not docs:
        print("因为没有加载到文件，所以建造程序无法启动...")
        return

    print("\n[步骤1/4] 正在把长篇档案切割成“知识卡片”...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(docs)
    print(f"-> 切割完成！我们得到了 {len(split_docs)} 张知识卡片！")

    print("\n[步骤2/4] 正在准备Embedding模型...")
    model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
    print("-> 装置已就绪！")

    print("\n[步骤3/4] 开始建造“语义书架”...")

    # 1. 先用第一张卡片初始化一个空的“书架”
    print("   -> 正在初始化书架结构...")
    first_doc_text = split_docs[0].page_content
    first_embedding = embeddings_model.embed_query(first_doc_text)
    vector_store = FAISS.from_embeddings([(first_doc_text, first_embedding)], embeddings_model, metadatas=[split_docs[0].metadata])

    # 2. 分批次地，把剩下的所有卡片加入到书架中
    batch_size = 32 # 每次处理32张卡片，这是一个比较高效的批量大小
    print(f"   -> 开始批量处理剩下的 {len(split_docs) - 1} 张卡片，批大小为 {batch_size}...")

    # tqdm会在这里创建一个漂亮的可视化进度条！
    for i in tqdm(range(1, len(split_docs), batch_size), desc="为知识卡片注入魔力并存入书架"):
        batch_docs = split_docs[i : i + batch_size]
        # 使用 add_documents 方法，这是最高效的批量添加方式
        vector_store.add_documents(batch_docs)

    print("\n-> “语义书架”建造成功！( ´ ▽ ` )ﾉ")

    print(f"\n[步骤4/4] 正在将“书架”永久保存在您的Google Drive里...")
    vector_store.save_local(save_path)
    print(f"\n🎉 任务圆满完成！我们最终的“语义书架”已经永久保存在: '{save_path}'")


# --- 主程序执行入口 ---
if __name__ == "__main__":
    # 挂载和路径设置
    drive.mount('/content/drive')
    project_path = "/content/drive/MyDrive/Akashi_Project"
    knowledge_base_path = os.path.join(project_path, "clean_knowledge")
    # 使用我们带标签的文件夹名
    VECTOR_STORE_PATH = os.path.join(project_path, "faiss_index_akashi_test")

    # 1. 加载并标记文档
    all_documents = load_and_tag_documents(knowledge_base_path)

    # 2. 调用带进度条的函数
    create_and_save_vector_store_with_progress(all_documents, VECTOR_STORE_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

 开始从您的Google Drive加载并分类知识水晶...


100%|██████████| 807/807 [00:16<00:00, 49.56it/s] 


  -> 从 'ships' 加载了 807 份档案，并标记为【角色人设】！


100%|██████████| 112/112 [00:00<00:00, 124.11it/s]


  -> 从 'Character' 加载了 112 份档案，并标记为【个人剧情】！


...

  -> 从 'SP' 加载了 75 份档案，并标记为【港区剧情】！

总共加载并标记了 1161 份档案！加载完成！♪

[步骤1/4] 正在把长篇档案切割成“知识卡片”...
-> 切割完成！我们得到了 12132 张知识卡片！

[步骤2/4] 正在准备Embedding模型...


/tmp/ipython-input-26722572.py:56: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

-> 装置已就绪！

[步骤3/4] 开始建造“语义书架”...
   -> 正在初始化书架结构...
   -> 开始批量处理剩下的 12131 张卡片，批大小为 32...


为知识卡片注入魔力并存入书架: 100%|██████████| 380/380 [1:45:40<00:00, 16.69s/it]



-> “语义书架”建造成功！( ´ ▽ ` )ﾉ

[步骤4/4] 正在将“书架”永久保存在您的Google Drive里...

🎉 任务圆满完成！我们最终的“语义书架”已经永久保存在: '/content/drive/MyDrive/Akashi_Project/faiss_index_akashi_test'


In [8]:
# ===============================================
# 第三部分: 加载并测试向量数据库
# ===============================================
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from google.colab import drive

# --- 准备阶段：确保Drive已连接并设置路径 ---
if not os.path.isdir('/content/drive/MyDrive'):
    drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Akashi_Project"
VECTOR_STORE_PATH = os.path.join(project_path, "faiss_index_akashi_test")

# --- 开始测试 ---
print("正在加载已构建的向量数据库...")

# 1. 初始化与构建时所用相同的嵌入模型
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

# 2. 从Google Drive加载向量数据库
if os.path.exists(VECTOR_STORE_PATH):
    db = FAISS.load_local(VECTOR_STORE_PATH, embeddings_model, allow_dangerous_deserialization=True)
    print("向量数据库加载成功。准备开始查询测试。")

    # 3. 定义一系列示例查询问题
    queries = [
        "阿蒂利奥和小兔子的交流",
        "介绍一下企业这位角色。",
        "什么是心智魔方",
        "阿尔萨斯是有怎样的性格",
        "布里有几个？"
    ]

    # 4. 遍历问题列表并执行搜索，展示结果
    for query in queries:
        print("\n" + "="*50)
        print(f"❓ [查询问题]: {query}")
        print("="*50)

        # k=3 参数指定返回3个最相关的结果
        results = db.similarity_search(query, k=3)

        if not results:
            print("   未能找到相关的文档片段。")
        else:
            print("💡 [数据库返回的最相关文档片段]:")
            for i, doc in enumerate(results):
                print(f"\n--- [相关片段 {i+1}] ---")
                # doc.metadata['source'] 包含来源文件名信息
                print(f"   来源文件: {os.path.basename(doc.metadata.get('source', '未知'))}")
                # doc.page_content 是文本片段的具体内容
                print(f"   片段内容:\n   {doc.page_content.strip().replace('\n', '\n   ')}")

else:
    print(f"错误：在指定路径 '{VECTOR_STORE_PATH}' 未找到向量数据库文件。请确认第二部分代码是否已成功执行。")


正在加载已构建的向量数据库...
向量数据库加载成功。准备开始查询测试。

❓ [查询问题]: 阿蒂利奥和小兔子的交流
💡 [数据库返回的最相关文档片段]:

--- [相关片段 1] ---
   来源文件: 091_主持人、艺人与秘书舰.md
   片段内容:
   顺便根据玩偶的原作者所说，这个既不是拉菲兔也不是本森兔，而是东煌玉兔，有象征团圆的含义在里面。
   
   喜欢么？
   
   关岛：
   
   喜欢~！象征团圆的兔子啊……我会好好摆在床头的！
   
   果然，相比起离别，还是团圆最好了！嘿嘿……
   
   少女的心情，正在雨过天晴。
   
   关岛：
   
   哇啊！糟糕，已经到这个时间了么！
   
   节目快开始了……没办法，随便找个地方看吧！
   
   向着公园，全速前进——！
   
   指挥官：
   
   欸，等……
   
   我话还没说完，就被少女拉着跑向了远方。
   
   玩了一天玩下来还能如此有活力……不愧是你，关岛。
   
   
   ## 两倍的真实
   
   随便找了个地方坐下后，关岛打开了随身平板。
   
   欢笑声从屏幕中传出，关岛所说的节目似乎已经开始了。
   
   港区·指挥室
   
   **茗：**
   > </span>
   指挥官，不好了喵！大事不好了喵！
   
   埃塞克斯：
   
   出什么事了，茗，为什么这么慌张？
   
   **茗：**
   > </span>
   港口外面，来了很多运送补给的货船喵！
   
   埃塞克斯：
   
   哦……算时间的话，差不多确实是到了送补给物资的时候了。
   
   **茗：**
   > </span>
   来的货轮足足有一百五十艘喵！
   
   埃塞克斯：

--- [相关片段 2] ---
   来源文件: REGULAR_596_阿蒂利奥·雷戈洛.md
   片段内容:
   * 指挥官！让兔子们来缓解你的疲劳~嘿！抱住你了不介意阿蒂利奥稍微占用一会你的时间吧？
     * 在指挥官身边时，阿蒂利奥就像公主一样！嘿嘿，最喜欢指挥官了~
   * **触摸台词：**
     * 要和阿蒂利奥还有小兔子们一起玩吗？
     * 嘿嘿，最喜欢和指挥官玩了~
  